In [ ]:
#!/usr/bin/env python3

# Python Script For All Tasks in Question B

import pyspark as pys
from pyspark.sql.functions import broadcast
#sparkC = pys.SparkContext()

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.master("spark://192.168.1.8:7077").config("Spark.Conf.Options", "random-value").getOrCreate()

spark_session.conf.set("spark.executor.memory", '4g')
spark_session.conf.set('spark.executor.cores', '3')
spark_session.conf.set('spark.cores.max', '3')
spark_session.conf.set("spark.driver.memory",'4g')
sparkC = spark_session.sparkContext
#ss = SparkSession.builder.appName("Python - Spark SQL basics").config("Spark.Conf.Options", "random-value").getOrCreate()

sic_codes_file1 = spark_session.read.csv("sic_codes_file.csv",header=True)
sic_codes_file1.createOrReplaceTempView("siccode")
sic_codes_file = spark_session.sql("SELECT * FROM siccode")

sic_codes_file_Min = spark_session.sql("SELECT Sector FROM siccode")

Data_A = spark_session.read.csv("data.csv",header=True)
Data_A.createOrReplaceTempView("people")

# Task B1.1
Data_A = spark_session.sql("SELECT DiffMeanHourlyPercent, EmployerName FROM people where DiffMeanHourlyPercent>=50 order by DiffMeanHourlyPercent desc")
Data_A.show()

Data_A = spark_session.sql("SELECT DiffMeanHourlyPercent, EmployerName FROM people where DiffMeanHourlyPercent>='0' and DiffMeanHourlyPercent<'10' order by DiffMeanHourlyPercent asc")
Data_A.show()

# Task B1.2
Data_A = spark_session.sql("SELECT sum(DiffMeanHourlyPercent)/10491 FROM people")
Data_A.show()

# Task B1.3
Data_A = spark_session.sql("SELECT sum(DiffMeanHourlyPercent)/10491 FROM people")
Data_A.toPandas().to_csv('Task_B_1_3.csv')

# Task B1.4
spark_session.sql("SELECT count(*) FROM people").show()

Data_A = spark_session.sql("SELECT count(*)/10491 FROM people where DiffMeanHourlyPercent<'0'")
Data_A.show()

# Task B2.1
#replace this code with an SQL query
Data_A=spark_session.sql("SELECT * FROM people")
#print(Data_A.SicCodes)
#print(sic_codes_file.Min)
#Data_A_SicCodes=ss.sql("SELECT SicCodes FROM people")
#cond= [(Data_A_SicCodes == sic_codes_file_Min) and (Data_A_SicCodes != 1)]
Data_B = Data_A.join(broadcast(sic_codes_file),((Data_A.SicCodes == sic_codes_file.Min) & (Data_A.SicCodes != 1)),"full_outer")
Data_B.show()
Data_C = Data_A.join(broadcast(sic_codes_file),((Data_A.SicCodes == sic_codes_file.Min) & (Data_A.SicCodes != 1)),"rightouter")
Data_C.show()
Data_B.toPandas().to_csv('Task_B_2_1.csv')
Data_C.toPandas().to_csv('Task_C_2_1.csv')
# Task B2.2
Data_B.createOrReplaceTempView("people2")
Data_C.createOrReplaceTempView("people3")

Data_B = spark_session.sql("SELECT (select sum(DiffMeanHourlyPercent) from people2) / (select count(Sector) FROM people3)")
Data_B.show()

# Task B2.3
Data_B = spark_session.sql("SELECT (select sum(DiffMedianHourlyPercent) FROM people2)/ (select count(Sector) FROM people3)")
Data_B.show()

Data_B = spark_session.sql("SELECT (select sum(DiffMeanBonusPercent) FROM people2)/ (select count(Sector) FROM people3)")
Data_B.show()

Data_B = ss.sql("SELECT (select sum(DiffMedianBonusPercent) FROM people2)/ (select count(Sector) FROM people3)")
Data_B.show()
sparkC.stop()